# Counting distinct values

It's extremely common for analysts to want to count unique occurences of some dimension value in data. With the Druid database's history of large volumes of data comes an advanced computer science technique to speed up this calculation through approximation. In this tutorial, work through some examples and see the effect of turning it on and off, and of making it even faster by pre-generating the objects that Druid uses to execute the query.

## Prerequisites

This tutorial works with Druid 26.0.0 or later.

#### Run using Docker

Launch this tutorial and all prerequisites using the `druid-jupyter` profile of the Docker Compose file for Jupyter-based Druid tutorials. For more information, see [Docker for Jupyter Notebook tutorials](https://druid.apache.org/docs/latest/tutorials/tutorial-jupyter-docker.html).
   
#### Run without using Docker

If you do not use the Docker Compose environment, you need the following:

* A running Apache Druid instance, with a `DRUID_HOST` local environment variable containing the servername of your Druid router
* [druidapi](https://github.com/apache/druid/blob/master/examples/quickstart/jupyter-notebooks/druidapi/README.md), a Python client for Apache Druid. Follow the instructions in the Install section of the README file.
* [matplotlib](https://matplotlib.org/), a library for creating visualizations in Python,
* [pandas](https://pandas.pydata.org/), a data analysis and manipulation tool.

### Initialize Python

Run the next cell to set up the Druid Python client's connection to Apache Druid.

If successful, the Druid version number will be shown in the output.

In [ ]:
import druidapi
import os

if 'DRUID_HOST' not in os.environ.keys():
    druid_host=f"http://localhost:8888"
else:
    druid_host=f"http://{os.environ['DRUID_HOST']}:8888"
    
print(f"Opening a connection to {druid_host}.")
druid = druidapi.jupyter_client(druid_host)

display = druid.display
sql_client = druid.sql
status_client = druid.status

status_client.version

### Load example data

Once your Druid environment is up and running, ingest the sample data for this tutorial.

Open the Druid console, and ingest the data as follows:

1. Select **Load data** from the top-level navigation.
2. Select **Batch - SQL**.
3. For the input type, select **Example data**.
4. Select **FlightCarrierOnTime (1 month)**.
5. Click **Use example**.

Go through the data loader wizard using the defaults, including the datasource name:

`On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11`

Run the following cell to describe the table, a handy way to check that the table that you will need for this notebook is present.

In [ ]:
display.table('On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11')

Finally, run the following cell to import additional Python modules that you will use.

In [ ]:
import json
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

## Using COUNT(DISTINCT)

This is a typical query you might run to find the number of distinct Tail Numbers in the example dataset.

```sql
SELECT
    "Reporting_Airline",
    COUNT(DISTINCT "Tail_Number") AS "Events"
FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
GROUP BY 1
ORDER BY 2
```

You're about to discover several ways enables you to arrive at this answer, each one having different performance characteristics depending on the needs of your users.

### COUNT(DISTINCT) with approximation

Druid automatically looks for query patterns that benefit from from approximation. In this instance, Druid identifies a match for approximate `COUNT(DISTINCT)`.

In Druid, each data server computes its own intermediate results that are then merged into a final result set. In a `COUNT(DISTINCT)` query, each process its own set of results, passes these back, and then computation is done to find the result. Without estimation, especially when the dimension being looked at contains many tens-of-thousands, perhaps even millions of unique values, the transfer of intermediate results and the final computation can be both memory and CPU hungry.

With approximation, the intermediate results are contained in a representation called a [data sketch](https://datasketches.apache.org/). These much smaller objects are transfered and merged together. Interestingly, data sketch size is not dependent on the underlying data – the default size of a sketch in Druid is just over 2000 bytes.

This translates into much faster query execution, especially when the intermediate results are large. The most scalable part of Druid – the data processes – do much more of the work, and do so earlier.

> Approximations improve scalability, storage, and memory use - at the cost of some error.
> 
> _[Gian Merlino](https://github.com/gianm)_

In your own deployment, check Druid's [configuration files](https://druid.apache.org/docs/26.0.0/configuration/index.html#sql) to see what type of data sketch is being used (`druid.sql.approxCountDistinct.function`) and whether this approach has been left as the default by your system administrators (`druid.sql.planner.useApproximateCountDistinct`).

Run the following cell to plot the number of events in the sample data for each `Tail_Number`. By default, each Druid process will be calculating its local results as data sketches, passing these back for the final merge operation on which size estimation is performed.

In [ ]:
sql = '''
SELECT
    "Reporting_Airline",
    COUNT(DISTINCT "Tail_Number") AS "Unique Tail Numbers"
FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
GROUP BY 1
ORDER BY 2
'''

df1 = pd.DataFrame(sql_client.sql(sql))

df1.plot.bar(x='Reporting_Airline', y='Unique Tail Numbers')
plt.show()

### COUNT(DISTINCT) without approximation

We can supply a query context parameter, `useApproximateCountDistinct`, to force Druid to not use this approximation technique.

Using the same SQL statement as before, the following cell crafts a request (`req`) where we set the context parameter before Druid executes the query.

In [ ]:
req = sql_client.sql_request(sql)
req.add_context("useApproximateCountDistinct", "false")
resp = sql_client.sql_query(req)

df2 = pd.DataFrame(resp.rows)
df2.plot.bar(x='Reporting_Airline', y='Unique Tail Numbers')
plt.show()

### Comparing approximate and non-approximate results

The next cell shows a comparison of the two results above: `df1` used the default approximation approach, while `df2` are the results where we turned approximation off.

In [ ]:
df3 = df1.compare(df2, keep_equal=True)
df3

The table shows:

* A row number
* The reporting airline in the approximate results (`self`) versus that in the non-approximate results (`other`)
* The calculated distinct number of `Tail Number`s

Notice that there are _value_ errors, as you might expect with approximation, and that in some instances this affects the _order_ of results.

Error in sketch-based approximation is probabilistic, rather than guaranteed. That's to say that a certain percentage of the time you can expect the measurements you take to be within a certain distance of the true value.

The results show, for each `Reporting_Airline`, the highly optimized, aggregated list of the `Tail_Number`s. By storing this in the table, Druid queries gains an overall speed boost from an increase in table data efficiency and a decrease in computation effort.

## Calculating set union with Theta and HyperLogLog sketches

There are two types of Apache Datasketch you can use to estimate the size of a union of one or more sets:

* [HyperLogLog](https://druid.apache.org/docs/26.0.0/querying/sql-aggregations.html#hll-sketch-functions)
* [Theta](https://druid.apache.org/docs/26.0.0/querying/sql-aggregations.html#theta-sketch-functions)

HyperLogLog and Theta sketches allow us to estimate the `COUNT(DISTINCT)` of the union of two or more sets. Consider that, when Druid executes a `COUNT(DISTINCT)` query in approximate mode, it is creating a set that is the union of independent sets of results - several from each and every data segment - and giving back to us an estimate the set size.

In Druid SQL, you have access to functions that allow you to define and union your own sets in order to estimate their size.

Run next cell, which:

* Gets three sets of `Tail_Number`s using `DS_HLL` - it applies a `FILTER` to isolate flights out of three specific cities,
* Applies `HLL_SKETCH_UNION` to union the three sets, and
* Estimates the resulting set size with `HLL_SKETCH_ESTIMATE`.

It uses `TIME_FLOOR` to giving us a week-by-week `GROUP BY` of the data.

In [ ]:
sql='''
SELECT
  TIME_FLOOR("__time",'P1W') AS "Week commencing",
  HLL_SKETCH_ESTIMATE(
     HLL_SKETCH_UNION(
       DS_HLL("Tail_Number_HLL") FILTER (WHERE "Origin"='ATL'),
       DS_HLL("Tail_Number_HLL") FILTER (WHERE "Origin"='DFW'),
       DS_HLL("Tail_Number_HLL") FILTER (WHERE "Origin"='SFO')
      )
    ) AS "AnyThreeCity-HLL",
  THETA_SKETCH_ESTIMATE(
     THETA_SKETCH_UNION(
       DS_THETA("Tail_Number_THETA") FILTER (WHERE "Origin"='ATL'),
       DS_THETA("Tail_Number_THETA") FILTER (WHERE "Origin"='DFW'),
       DS_THETA("Tail_Number_THETA") FILTER (WHERE "Origin"='SFO')
      )
    ) AS "AnyThreeCity-THETA"
FROM "flights-counts"
WHERE TIMESTAMP '2005-10-31' <= __time AND __time <= TIMESTAMP '2005-11-20'
GROUP BY 1
'''

display.sql(sql)

## Calculating set intersection and difference with Theta sketches

With Theta sketches, you can also approximate the size of:

* The intersection of two sets (people who went to both MoMa _and_ the NPG)
* The difference between one set and another (people who went to MoMa _not_ the NPG)

For simplicity, the following sections contain queries that take advantage of the data set you've created that contains sketches. You can apply the same techniques on tables without sketches in them – use `DS_THETA` function on the raw data instead.

### Set intersection

Run the next cell to see set difference being used with Theta sketches to produce a count estimate of the overlap between the sets.

### Set difference

Finally, run the next cell to use Theta sketch operations to estimate the size of the difference between one set and another.

Note that this operation is not cumutative - Druid calculates the size of the difference (A to B), not symetric difference, between the sets.

## Conclusion

* Approximation is the default execution model for `COUNT(DISTINCT)` queries
* You can turn it off with a query context parameter
* Accuracy is highly dependent on the distribution and cardinality of data across the database
* Druid can be pre-loaded with sketch objects that speed up approximation both in batch and streaming ingestion
* HyperLogLog and Theta sketches both allow you to approximate `COUNT(DISTINCT)` of entire sets
* Only Theta sketches allow you to carry out set operations

## Learn more

* Try estimation on your own dataset:
    * Identify a high-cardinality column in one of your own data sets
    * Test how long an approximate `DISTINCT(COUNT)` query takes to run with approximation turned on
    * Test how long the same query takes to run with approximation turned off
* Watch [Employ Approximation](https://youtu.be/fSWwJs1gCvQ?list=PLDZysOZKycN7MZvNxQk_6RbwSJqjSrsNR) by Peter Marshall
* Read [Ingesting Data Sketches into Apache Druid](https://blog.hellmar-becker.de/2022/12/26/ingesting-data-sketches-into-apache-druid/) by Hellmar Becker
* Read more about the native "aggregator" functions for streaming ingestion
    * [ThetaSketch function](https://druid.apache.org/docs/26.0.0/development/extensions-core/datasketches-theta.html)
    * [HyperLogLog function](https://druid.apache.org/docs/26.0.0/development/extensions-core/datasketches-hll.html)